# Mise en œuvre d'une classe de propriétés personnalisées

Dans le chapitre précédent de notre tutoriel, nous avons appris à créer et à utiliser des propriétés dans une classe. L'objectif principal était de les comprendre comme un moyen de se débarrasser des getters et setters explicites et d'avoir une interface de classe simple. Cette connaissance est généralement suffisante pour la plupart des programmeurs et pour les cas d'utilisation pratiques, et ils n'auront pas besoin de plus.

Si vous voulez en savoir plus sur le fonctionnement des __propriétés__, vous pouvez aller plus loin avec nous. Vous pourrez ainsi améliorer vos compétences en codage et approfondir votre compréhension de Python. Nous allons examiner la manière dont le décorateur __property__ pourrait être implémenté dans le code Python. (En réalité, il est implémenté en code C !) En faisant cela, la façon de travailler sera plus claire. Tout est basé sur le protocole des descripteurs, que nous expliquerons plus tard.

<center><img src="img/illustration3.png" width="50%"></center>

Nous définissons une classe avec le nom ```our_property``` afin qu'elle ne soit pas confondue avec la classe "propriété" existante. Cette classe peut être utilisée comme la "vraie" classe de propriété.

In [2]:
class our_property:
    """ emulation of the property class 
        for educational purposes """

    def __init__(self, 
                 fget=None, 
                 fset=None, 
                 fdel=None, 
                 doc=None):
        """Attributes of 'our_decorator'
        fget
            function to be used for getting 
            an attribute value
        fset
            function to be used for setting 
            an attribute value
        fdel
            function to be used for deleting 
            an attribute
        doc
            the docstring
        """
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

Nous avons besoin d'une autre classe pour utiliser la classe précédemment définie et pour démontrer le fonctionnement du décorateur de classe de propriété. Pour continuer la tradition des chapitres précédents de notre tutoriel Python, nous allons à nouveau écrire une classe Robot. Nous allons définir une propriété dans cette classe d'exemple pour démontrer l'utilisation de notre classe de propriété précédemment définie ou mieux 'notre_décorateur'. Lorsque vous exécutez le code, vous pouvez voir que ```__init__``` de ```our_property``` sera appelé ```fget``` avec une référence à la fonction ```getter``` de ```city```. L'attribut ```city``` est une instance de la classe ```our_property```. La classe ```our_property``` fournit un autre décorateur ```setter``` pour la fonction ```setter```. Nous l'appliquons avec ```@city.setter```.

In [3]:
class Robot:
    
    def __init__(self, city):
        self.city = city
        
    @our_property
    def city(self):
        print("The Property 'city' will be returned now:")
        return self.__city
    
    @city.setter
    def city(self, city):
        print("'city' will be set")
        self.__city = city

```Robot.city``` est une instance de la classe ```our_property``` comme nous pouvons le voir ci-après :

In [4]:
type(Robot.city)

__main__.our_property

Si vous remplacez la ligne ```@our_property``` par ```@property```, le programme se comportera de la même manière, mais il utilisera la classe Python originale ```property``` et non notre classe ```our_property```. Nous allons créer des instances de la classe Robot dans le code Python suivant :

In [6]:
print("Instantiating a Root and setting 'city' to 'Berlin'")
robo = Robot("Berlin")
print("The value is: ", robo.city)
print("Our robot moves now to Frankfurt:")
robo.city = "Frankfurt"
print("The value is: ", robo.city)

Instantiating a Root and setting 'city' to 'Berlin'
'city' will be set
The Property 'city' will be returned now:
The value is:  Berlin
Our robot moves now to Frankfurt:
'city' will be set
The Property 'city' will be returned now:
The value is:  Frankfurt


Rendons notre implémentation de propriété un peu plus bavarde avec quelques fonctions d'impression pour voir ce qui se passe. Nous changeons également le nom en ```chatty_property``` pour des raisons évidentes :

In [7]:
class chatty_property:
    """ emulation of the property class 
        for educational purposes """

    def __init__(self, 
                 fget=None, 
                 fset=None, 
                 fdel=None, 
                 doc=None):
        
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        print("\n__init__ called with:)")
        print(f"fget={fget}, fset={fset}, fdel={fdel}, doc={doc}")
        if doc is None and fget is not None:
            print(f"doc set to docstring of {fget.__name__} method")
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        print(type(self))
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [8]:
class Robot:
    
    def __init__(self, city):
        self.city = city
        
    @chatty_property
    def city(self):
        """ city attribute of Robot """
        print("The Property 'city' will be returned now:")
        return self.__city
    
    @city.setter
    def city(self, city):
        print("'city' will be set")
        self.__city = city


__init__ called with:)
fget=<function Robot.city at 0x10f666700>, fset=None, fdel=None, doc=None
doc set to docstring of city method
<class '__main__.chatty_property'>

__init__ called with:)
fget=<function Robot.city at 0x10f666700>, fset=<function Robot.city at 0x10f6667a0>, fdel=None, doc= city attribute of Robot 


In [9]:
robo = Robot("Berlin")

'city' will be set
